In [1]:
import numpy as np
import pandas as pd
from scipy import stats

import os
import re

#from google.colab import drive
#drive.mount('/content/gdrive')
os.chdir('C:\insta\data')

# Import "orders.csv"

In [2]:
# import "orders.csv"
#df_orders = pd.read_csv('gdrive/My Drive/Colab Notebooks/instacart/orders.csv')
df_orders = pd.read_csv('orders.csv')
print(df_orders.shape)    # (3421083, 7)
df_orders.head(5)

# fill "NA" with 0
df_orders.columns[df_orders.isnull().any()]                   # column "days_since_prior_order" has missing values
df_orders['days_since_prior_order'].fillna(0, inplace=True)   # replace that missing value with 0
print(df_orders.isnull().values.any())                                 # false

# seperate "orders" into 3 dataframes based on the value of "eval_set"
df_orders_prior = df_orders.loc[df_orders['eval_set'] == 'prior']      # data for creating features
df_orders_prior.shape   # (3214874, 7)

df_orders_train = df_orders.loc[df_orders['eval_set'] == 'train']      # data for training 
df_orders_test = df_orders.loc[df_orders['eval_set'] == 'test']        # data for test

(3421083, 7)
False


# Import "order_products__prior.csv"

In [3]:
#df_order_products__prior = pd.read_csv('gdrive/My Drive/Colab Notebooks/instacart/order_products__prior.csv')
df_order_products__prior = pd.read_csv('order_products__prior.csv')
print(df_order_products__prior.shape)    # (32434489, 4)
df_order_products__prior.head(5)

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


# Merge "order_products__prior.csv" with "orders.csv" on 'order_id'

In [4]:
df_orders_prior_copy = df_orders_prior.loc[:, ['order_id','user_id','order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']]
df_prior_merged = pd.merge(df_order_products__prior, df_orders_prior_copy, on = 'order_id')
df_prior_merged.head(5)
#df_prior_merged[df_prior_merged.user_id == 1]
#df_prior_merged.to_csv('gdrive/My Drive/Colab Notebooks/instacart/df_prior_merged.csv', index = False)

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,3,5,9,8.0
1,2,28985,2,1,202279,3,5,9,8.0
2,2,9327,3,0,202279,3,5,9,8.0
3,2,45918,4,1,202279,3,5,9,8.0
4,2,30035,5,0,202279,3,5,9,8.0


# Import "products.csv"

In [5]:
#df_products = pd.read_csv('gdrive/My Drive/Colab Notebooks/instacart/products.csv')
df_products = pd.read_csv('products.csv')
#df_products.set_index('product_id', inplace=True)
#df_products_copy = df_products.copy()

print(df_products.shape)    # (49688, 3)
df_products.head(5)

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


# Create features based on each product
Add order numbers of each product, reorder times, and reorder rate to "products.csv"

In [6]:
order_per_prod = pd.DataFrame()
order_per_prod['prod_num_orders'] = df_order_products__prior.groupby(df_order_products__prior.product_id).size().astype(np.int32)  # how many orders include the product_id
order_per_prod['prod_reorder_times'] = df_order_products__prior['reordered'].groupby(df_order_products__prior.product_id).sum().astype(np.int32)  # reorder times for each product_id
order_per_prod['prod_reorder_rate'] = (order_per_prod['prod_reorder_times']/order_per_prod['prod_num_orders']).astype(np.float32)   # reorder rate
order_per_prod = order_per_prod.reset_index()
print(order_per_prod.head(5))

df_products_merged = pd.merge(df_products, order_per_prod, on = 'product_id')
del order_per_prod
df_products_merged.shape   #(49677, 6)
 
# average days between the orders  
temp_prod_avg_interval = df_prior_merged.groupby('product_id')['days_since_prior_order'].aggregate('mean').to_frame('prod_avg_interval').astype(np.float16)  
temp_prod_avg_interval = temp_prod_avg_interval.reset_index()

# standard deviation of purchase date (dow) 
temp_prod_dow_std =  df_prior_merged.groupby('product_id')['order_dow'].aggregate('std').to_frame('prod_dow_std').astype(np.float16) 
temp_prod_dow_std = temp_prod_dow_std.reset_index()

# standard deviation of purchase hour
temp_prod_hour_std =  df_prior_merged.groupby('product_id')['order_hour_of_day'].aggregate('std').to_frame('prod_hour_std').astype(np.float16) 
temp_prod_hour_std = temp_prod_hour_std.reset_index()
    
# mean of dow among all the orders 
temp_prod_dow_median =  df_prior_merged.groupby('product_id')['order_dow'].aggregate('median').to_frame('prod_dow_median').astype(np.float16) 
temp_prod_dow_median = temp_prod_dow_median.reset_index()

# mean of hour among all the orders 
temp_prod_hour_median =  df_prior_merged.groupby('product_id')['order_hour_of_day'].aggregate('median').to_frame('prod_hour_median').astype(np.float16) 
temp_prod_hour_median = temp_prod_hour_median.reset_index()

   product_id  prod_num_orders  prod_reorder_times  prod_reorder_rate
0           1             1852                1136           0.613391
1           2               90                  12           0.133333
2           3              277                 203           0.732852
3           4              329                 147           0.446809
4           5               15                   9           0.600000


In [7]:
# Day of week this product was  purchased most frequently
temp_prod_dow = df_prior_merged.groupby(by=['product_id'])['order_dow'].agg(lambda x : stats.mode(x)[0][0]).to_frame('prod_dow_most')  
temp_prod_dow = temp_prod_dow.reset_index()

# Time of day that this product was purchased most frequently
temp_prod_hour = df_prior_merged.groupby(by=['product_id'])['order_hour_of_day'].agg(lambda x : stats.mode(x)[0][0]).to_frame('prod_hour_most') 
temp_prod_hour = temp_prod_hour.reset_index()

df_products_merged = df_products_merged.merge(temp_prod_avg_interval, on = 'product_id', how = 'left')
df_products_merged = df_products_merged.merge(temp_prod_dow_std, on = 'product_id', how = 'left')
df_products_merged = df_products_merged.merge(temp_prod_hour_std, on = 'product_id', how = 'left')
df_products_merged = df_products_merged.merge(temp_prod_dow_median, on = 'product_id', how = 'left')
df_products_merged = df_products_merged.merge(temp_prod_hour_median, on = 'product_id', how = 'left')
df_products_merged = df_products_merged.merge(temp_prod_dow, on = 'product_id', how = 'left')
df_products_merged = df_products_merged.merge(temp_prod_hour, on = 'product_id', how = 'left')
del temp_prod_avg_interval
del temp_prod_dow_std
del temp_prod_hour_std
del temp_prod_dow_median
del temp_prod_hour_median
del temp_prod_dow
del temp_prod_hour

#df_products_merged.to_csv('gdrive/My Drive/Colab Notebooks/instacart/df_products_merged.csv', index = False)
df_products_merged.to_csv('df_products_merged.csv', index = False)
df_products_merged.head(5)

,product_id,product_name,aisle_id,department_id,prod_num_orders,prod_reorder_times,prod_reorder_rate,prod_avg_interval,prod_dow_std,prod_hour_std,prod_dow_median,prod_hour_median,prod_dow_most,prod_hour_most
0,1,Chocolate Sandwich Cookies,61,19,1852,1136,0.613391,9.882812,1.825195,4.253906,3.0,13.0,1,10
1,2,All-Seasons Salt,104,13,90,12,0.133333,10.132812,2.152344,3.941406,3.0,13.0,0,11
2,3,Robust Golden Unsweetened Oolong Tea,94,7,277,203,0.732852,9.914062,2.205078,4.515625,2.0,12.0,1,11
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,329,147,0.446809,12.945312,2.156250,4.371094,2.0,14.0,0,14
4,5,Green Chile Anytime Sauce,5,13,15,9,0.600000,11.601562,2.283203,4.546875,3.0,11.0,0,11


# Create features based on each user

In [8]:
df_user = pd.DataFrame()
df_user['user_num_orders'] = df_orders_prior.groupby('user_id').size().astype(np.int16)                     # total number of orders
df_user['user_num_prods'] = df_prior_merged.groupby('user_id').size().astype(np.int16)                      # total number of products
df_user['user_all_prods_names'] = df_prior_merged.groupby('user_id')['product_id'].apply(set)               # all products' names a user bought
df_user['user_num_distinct_prods'] = df_user['user_all_prods_names'].map(len).astype(np.int16)              # number of distinct products a user bought
df_user['user_avg_num_prods'] = df_user['user_num_prods']/df_user['user_num_orders']                        # average number of products in each order

df_user['user_avg_interval'] = df_orders_prior.groupby('user_id')['days_since_prior_order'].mean().astype(np.float16)   # average days between the orders

df_user['user_dow_std'] = df_orders_prior.groupby('user_id')['order_dow'].std().astype(np.float16)  # standard deviation of purchase date (dow) 
df_user['user_hour_std'] = df_orders_prior.groupby('user_id')['order_hour_of_day'].std().astype(np.float16)   # standard deviation of purchase hour
df_user['user_dow_median'] =  df_orders_prior.groupby('user_id')['order_dow'].median().astype(np.float16)             # mean of dow among all the orders
df_user['user_hour_median'] =  df_orders_prior.groupby('user_id')['order_hour_of_day'].median().astype(np.float16)     # mean of hour among all the orders
df_user = df_user.reset_index()
df_user.head(5)

,user_id,user_num_orders,user_num_prods,user_all_prods_names,user_num_distinct_prods,user_avg_num_prods,user_avg_interval,user_dow_std,user_hour_std,user_dow_median,user_hour_median
0,1,10,59,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",18,5.900000,17.593750,1.269531,3.591797,2.5,8.5
1,2,14,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",102,13.928571,14.140625,1.231445,1.554688,2.0,10.0
2,3,12,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33,7.333333,11.085938,1.311523,1.443359,0.5,16.0
3,4,5,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17,3.600000,11.000000,0.836426,1.672852,5.0,13.0
4,5,4,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",23,9.250000,10.000000,1.500000,2.828125,2.0,17.0


In [9]:
temp_user_dow = df_orders_prior.groupby(by=['user_id'])['order_dow'].agg(lambda x : stats.mode(x)[0][0]).to_frame('user_dow_most')  # Day of week that this user is most likely to place an order
temp_user_hour = df_orders_prior.groupby(by=['user_id'])['order_hour_of_day'].agg(lambda x : stats.mode(x)[0][0]).to_frame('user_hour_most') # Time of day that this user os most likely to place an order
temp_user_dow = temp_user_dow.reset_index()
temp_user_hour = temp_user_hour.reset_index()
df_user = df_user.merge(temp_user_dow, on = 'user_id', how = 'left')
df_user = df_user.merge(temp_user_hour, on = 'user_id', how = 'left')
del temp_user_dow
del temp_user_hour

temp_lastest_ordernum = df_prior_merged.groupby('user_id')['order_number'].aggregate('max').reset_index()  # the last order number of a user
temp_df_prior_last = pd.merge(df_prior_merged, temp_lastest_ordernum, how = 'inner', on = ['user_id', 'order_number'])

df_user['user_num_prods_last_order'] = temp_df_prior_last.groupby('user_id').size().astype(np.int16)      # count the number of products in the last order

temp_user_reorder_ratio = df_prior_merged.groupby(by = 'user_id')['reordered'].aggregate('mean').to_frame('user_reorder_ratio').astype(np.float16) # user reorder ration among all products
temp_user_reorder_ratio = temp_user_reorder_ratio.reset_index()

df_user = df_user.merge(temp_user_reorder_ratio, on = 'user_id', how = 'left')
del temp_user_reorder_ratio

df_user = df_user.drop(['user_all_prods_names'], axis = 1)
#df_user.to_csv('gdrive/My Drive/Colab Notebooks/instacart/df_user.csv', index = False)
df_user.to_csv('df_user.csv', index = False)
df_user.head(5)

,user_id,user_num_orders,user_num_prods,user_num_distinct_prods,user_avg_num_prods,user_avg_interval,user_dow_std,user_hour_std,user_dow_median,user_hour_median,user_dow_most,user_hour_most,user_num_prods_last_order,user_reorder_ratio
0,1,10,59,18,5.900000,17.593750,1.269531,3.591797,2.5,8.5,1,7,NaN,0.694824
1,2,14,195,102,13.928571,14.140625,1.231445,1.554688,2.0,10.0,1,10,9.0,0.476807
2,3,12,88,33,7.333333,11.085938,1.311523,1.443359,0.5,16.0,0,16,16.0,0.625000
3,4,5,18,17,3.600000,11.000000,0.836426,1.672852,5.0,13.0,4,11,6.0,0.055542
4,5,4,37,23,9.250000,10.000000,1.500000,2.828125,2.0,17.0,3,18,3.0,0.378418


# Create features based on each user and each product

In [10]:
# reorder status of each user and each product among all orders
#temp_reorder_status = df_prior_merged.groupby(['user_id','product_id'])['reordered'].aggregate('max').reset_index() 
#temp_reorder_status.head(5)

# get the reorder status of the last order
#temp_lastest_ordernum = df_prior_merged.groupby('user_id')['order_number'].aggregate('max').reset_index()  # total order numbers of a user
#temp_df_prior_last = pd.merge(df_prior_merged, temp_lastest_ordernum, how = 'inner', on=['user_id', 'order_number'])   
temp_df_latest_status = temp_df_prior_last[['user_id', 'product_id', 'reordered']]             
temp_df_latest_status.columns = ['user_id', 'product_id', 'spec_last_reorder_status'] 

# calculate how many orders include this product and Reorder times of this prouct 
df_user_product = df_prior_merged.groupby(['user_id','product_id'])['reordered'].aggregate(['count', 'sum']).reset_index()
df_user_product.columns = ['user_id', 'product_id', 'spec_num_order', 'spec_num_reorder']

# calculate % of this user's orders that include this product
temp_df_total_num_order = df_user[['user_id', 'user_num_orders']]    # get user_id, total number of orders for each user
df_user_product = pd.merge(df_user_product, temp_df_total_num_order, how = 'left', on = 'user_id')
df_user_product['spec_perc_order'] = (df_user_product['spec_num_order']/df_user_product['user_num_orders']).astype(np.float16) 

# The order number when the user has bought a product the first time
temp_spec_first_order_num = df_prior_merged.groupby(by=['user_id', 'product_id'])['order_number'].aggregate('min').to_frame('spec_first_order_num').reset_index() 
temp_spec_first_order_num.head(5)
df_user_product = pd.merge(df_user_product, temp_spec_first_order_num, how = 'left', on = ['user_id','product_id'])
del temp_spec_first_order_num

# The order range between the last order number and the first order number
df_user_product['spec_order_range'] =  df_user_product['user_num_orders'] - df_user_product['spec_first_order_num'] + 1

# order ratio among the order range
df_user_product['spec_order_ratio'] = (df_user_product['spec_num_order']/df_user_product['spec_order_range']).astype(np.float16) 
del df_user_product['user_num_orders']

# calculate the reorder status in the last order
#df_user_product = pd.merge(df_user_product, temp_reorder_status, how = 'inner', on=['user_id', 'product_id'])
df_user_product = pd.merge(df_user_product, temp_df_latest_status, how = 'left', on=['user_id','product_id'])
df_user_product['spec_last_reorder_status'].fillna(0, inplace = True)   # replace that missing value with 0

df_user_product.columns = ['user_id', 'product_id', 'spec_num_order', 'spec_num_reorder', 'spec_perc_order',
                           'spec_first_order_num', 'spec_order_range', 'spec_order_ratio', 'spec_last_reorder_status',]
                           
df_user_product = df_user_product[['user_id', 'product_id', 'spec_last_reorder_status', 'spec_num_order',  'spec_num_reorder', 'spec_perc_order', \
                                  'spec_first_order_num', 'spec_order_range', 'spec_order_ratio']]

# calculate rank # of this product for this user (most reordered, second most reordered, etc)
df_user_product['spec_popularity_prod'] = df_user_product.groupby('user_id')['spec_num_reorder'].rank(ascending = True, method ='dense').astype(np.int16)
df_user_product.head(5)

,user_id,product_id,spec_last_reorder_status,spec_num_order,spec_num_reorder,spec_perc_order,spec_first_order_num,spec_order_range,spec_order_ratio,spec_popularity_prod
0,1,196,1.0,10,9,1.000000,1,10,1.000000,6
1,1,10258,1.0,9,8,0.899902,2,9,1.000000,5
2,1,10326,0.0,1,0,0.099976,5,6,0.166626,1
3,1,12427,1.0,10,9,1.000000,1,10,1.000000,6
4,1,13032,1.0,3,2,0.300049,2,9,0.333252,3


In [11]:
# Average sequence when this product was added in the cart
temp_spec_avg_cart_seq = df_prior_merged.groupby(['user_id','product_id'])['add_to_cart_order'].aggregate('mean').to_frame('spec_avg_cart_seq').astype(np.float16)
temp_spec_avg_cart_seq = temp_spec_avg_cart_seq.reset_index()

# Average days between the orders that include this product
temp_spec_avg_interval = df_prior_merged.groupby(['user_id','product_id'])['days_since_prior_order'].aggregate('mean').to_frame('spec_avg_interval').astype(np.float16)
temp_spec_avg_interval = temp_spec_avg_interval.reset_index()

# Day of week that this user is most likely to buy this product
temp_spec_dow_median = df_prior_merged.groupby(['user_id','product_id'])['order_dow'].aggregate('median').to_frame('spec_dow_median').astype(np.float16)
temp_spec_dow_median = temp_spec_dow_median.reset_index()

# Time of day that this user is most likely to buy this product 
temp_spec_hour_median = df_prior_merged.groupby(['user_id','product_id'])['order_hour_of_day'].aggregate('median').to_frame('spec_hour_median').astype(np.float16)
temp_spec_hour_median = temp_spec_hour_median.reset_index()

df_user_product = pd.merge(df_user_product, temp_spec_avg_cart_seq, how = 'left', on = ['user_id', 'product_id'])
df_user_product = pd.merge(df_user_product, temp_spec_avg_interval, how = 'left', on = ['user_id', 'product_id'])
df_user_product = pd.merge(df_user_product, temp_spec_dow_median, how = 'left', on = ['user_id', 'product_id'])
df_user_product = pd.merge(df_user_product, temp_spec_hour_median, how = 'left', on = ['user_id', 'product_id'])

del temp_spec_avg_cart_seq
del temp_spec_avg_interval
del temp_spec_dow_median
del temp_spec_hour_median

# Reverse the order number for each user 
df_prior_merged['order_num_back'] = df_prior_merged.groupby(by=['user_id'])['order_number'].transform(max) - df_prior_merged.order_number + 1
temp_last5 = df_prior_merged.loc[df_prior_merged.order_num_back <= 5]  # filter the last 5 orders
# How many times a customer bought a product on its last 5 orders
temp_spec_num_last5 = temp_last5.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('spec_num_last5').reset_index()

df_user_product = pd.merge(df_user_product, temp_spec_num_last5, how = 'left', on = ['user_id', 'product_id'])
del temp_spec_num_last5

# ratio of the product bought in the last_five orders
df_user_product['spec_order_ratio_last5'] = (df_user_product['spec_num_last5']/5).astype(np.float16)

df_user_product.spec_num_last5.fillna(0, inplace = True)
df_user_product.spec_order_ratio_last5.fillna(0, inplace = True)

#df_user_product.to_csv('gdrive/My Drive/Colab Notebooks/instacart/df_user_product.csv', index = False)
df_user_product.to_csv('df_user_product.csv', index = False)
df_user_product.head(5)

,user_id,product_id,spec_last_reorder_status,spec_num_order,spec_num_reorder,spec_perc_order,spec_first_order_num,spec_order_range,spec_order_ratio,spec_popularity_prod,spec_avg_cart_seq,spec_avg_interval,spec_dow_median,spec_hour_median,spec_num_last5,spec_order_ratio_last5
0,1,196,1.0,10,9,1.000000,1,10,1.000000,6,1.400391,17.593750,2.5,8.5,5.0,1.000000
1,1,10258,1.0,9,8,0.899902,2,9,1.000000,5,3.333984,19.562500,3.0,9.0,5.0,1.000000
2,1,10326,0.0,1,0,0.099976,5,6,0.166626,1,5.000000,28.000000,4.0,15.0,0.0,0.000000
3,1,12427,1.0,10,9,1.000000,1,10,1.000000,6,3.300781,17.593750,2.5,8.5,5.0,1.000000
4,1,13032,1.0,3,2,0.300049,2,9,0.333252,3,6.332031,21.671875,3.0,8.0,2.0,0.399902


# Create features based on dow (day of a week)

In [12]:
df_dow = pd.DataFrame()
df_dow['t_dow_orders'] = df_prior_merged.groupby('order_dow').size().astype(np.int32)
#df_dow.to_csv('gdrive/My Drive/Colab Notebooks/instacart/df_dow.csv')
df_dow.to_csv('df_dow.csv')
df_dow.head(5)


,t_dow_orders
order_dow,
0,6209666
1,5665856
2,4217798
3,3844117
4,3787215


# Create features based on hour


In [13]:
df_hour = pd.DataFrame()
df_hour['t_hour_orders'] = df_prior_merged.groupby('order_hour_of_day').size().astype(np.int32)
#df_hour.to_csv('gdrive/My Drive/Colab Notebooks/instacart/df_hour.csv')
df_hour.to_csv('df_hour.csv')
df_hour.head(5)

,t_hour_orders
order_hour_of_day,
0,218948
1,115786
2,69434
3,51321
4,53283
